### Постановка задачи

Просмотр фильмов на оригинальном языке - это популярный и действенный метод обучения иностранным языкам. Важно выбрать фильм, который подходит студенту по уровню сложности, т.е. студент понимал бы 50-70 % диалогов. Чтобы выполнить это условие, преподаватель должен посмотреть фильм и решить, какому уровню он соответствует. Однако это требует больших временных затрат.

Наша задача - разработать ML решение для автоматического определения уровня сложности англоязычных фильмов. 

Программа-максимум:

           - языковая модель, 
           - веб-интерфейс,
           - микросервис.  
           
### Знакомство с исходными данными

В качестве исходных данных нам предоставлены размеченные по уровню сложности фильмы и субтитры к ним, словари английского языка с набором слов по уровню.

Сначала взглянем на эти файлы, чтоб понять, как можно было бы с ними вообще работать.

In [1]:
import pysrt

In [2]:
path_A2 = './English_level/English scores/Subtitles_all/A2/The Walking Dead-S01E01-Days Gone Bye.English.srt'

In [3]:
subs = pysrt.open(path_A2)

In [4]:
len(subs)

673

In [5]:
for i in range(len(subs)):
    print(subs[i])

1
00:00:03,169 --> 00:00:05,171
( bugs chittering )

2
00:00:26,859 --> 00:00:28,861
( brakes squeak )

3
00:00:30,696 --> 00:00:34,158
- ( engine stops )
- ( trunk clicks )

4
00:00:37,954 --> 00:00:39,956
( bird cawing )

5
00:01:33,551 --> 00:01:35,553
( birds chirping )

6
00:02:12,381 --> 00:02:14,383
( flies buzzing )

7
00:02:19,639 --> 00:02:21,641
( metal rattling )

8
00:02:45,790 --> 00:02:47,792
( shuffling footsteps )

9
00:03:29,458 --> 00:03:31,210
Little girl?

10
00:03:32,461 --> 00:03:34,337
I'm a policeman.

11
00:03:34,504 --> 00:03:36,589
Little girl.

12
00:03:40,552 --> 00:03:42,721
Don't be afraid, okay?

13
00:03:45,514 --> 00:03:47,390
Little girl.

14
00:03:56,399 --> 00:03:58,401
( panting )

15
00:03:59,736 --> 00:04:01,738
( growling )

16
00:04:06,325 --> 00:04:08,077
Oh my God.

17
00:04:19,881 --> 00:04:22,884
( theme music playing )

18
00:04:59,796 --> 00:04:59,921
- ( police radio chatter )
- Man: What's the difference

19
00:04:59,921 --> 00:05:01,6

Чтобы проанализировать слова, нужно из этой структуры убрать тайминг, нумерацию кадров (видимо, это строки, потому что их количество равно длине файла с субтитрами), убрать слова в скобках, потому что в задаче запрос на понимание диалогов, а в скобках не диалоги, убрать знаки препинания и цифры, привести к нижнему регистру.

In [6]:
#import pyprind
import pandas as pd
import os


basepath = './English_level/English scores/Subtitles_all/'


#pbar = pyprind.ProgBar(50000)
df_1 = pd.DataFrame()
for s in ('A2', 'B1', 'B2', 'C1'):
    #print(s)
    path = os.path.join(basepath, s)
    for file in os.listdir(path):
        #print(file)
        with open(os.path.join(path, file), 'r', encoding='ISO-8859-1') as infile:
            txt = infile.read()
        df_1 = df_1.append([[file[:-4], s, txt]], ignore_index=True)
        #pbar.update()
df_1.columns = ['Movie', 'Level', 'Srt']

C:\Users\osen1\AppData\Local\Temp\ipykernel_4376\1577275278.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1 = df_1.append([[file[:-4], s, txt]], ignore_index=True)


In [7]:
display(df_1)

,Movie,Level,Srt
0,The Walking Dead-S01E01-Days Gone Bye.English,A2,"1\n00:00:03,169 --> 00:00:05,171\n( bugs chitt..."
1,The Walking Dead-S01E02-Guts.English,A2,"1\n00:00:03,045 --> 00:00:05,047\n- ( birds ch..."
2,The Walking Dead-S01E03-Tell It To The Frogs.E...,A2,"1\n00:00:03,003 --> 00:00:04,671\n( thunder ru..."
3,The Walking Dead-S01E04-Vatos.English,A2,"1\n00:00:03,045 --> 00:00:05,422\n( birds chir..."
4,The Walking Dead-S01E05-Wildfire.English,A2,"1\n00:00:03,420 --> 00:00:04,922\n- ( walkie-t..."
...,...,...,...
158,Suits.S03E06.720p.HDTV.x264-mSD,C1,"ï»¿1\n00:00:01,383 --> 00:00:02,751\nI lost Av..."
159,Suits.S03E07.HDTV.x264-mSD,C1,"ï»¿1\n00:00:00,052 --> 00:00:01,352\nPreviousl..."
160,Suits.S03E08.480p.HDTV.x264-mSD,C1,"ï»¿1\n00:00:01,436 --> 00:00:03,028\nI get Ava..."
161,Suits.S03E09.480p.HDTV.x264-mSD,C1,"ï»¿1\n00:00:00,024 --> 00:00:01,478\nPreviousl..."


В первой таблице, собранной из данных папки заказчика с распределением субтитров по уровню языка,  у нас есть полный набор данных - название фильма, его рейтинг, текст субтитров.

In [8]:
#pbar = pyprind.ProgBar(50000)
df_2 = pd.DataFrame()
path = os.path.join(basepath, 'Subtitles')
for file in os.listdir(path):
    if file!='.DS_Store':
        #print(file)
        with open(os.path.join(path, file), 'r', encoding='ISO-8859-1') as infile:
            txt = infile.read()
        df_2 = df_2.append([[file[:-4], txt]], ignore_index=True)
        #pbar.update()
df_2.columns = ['Movie', 'Srt']

C:\Users\osen1\AppData\Local\Temp\ipykernel_4376\1041165742.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2 = df_2.append([[file[:-4], txt]], ignore_index=True)


In [9]:
display(df_2)

,Movie,Srt
0,10_Cloverfield_lane(2016),"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,10_things_I_hate_about_you(1999),"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,Aladdin(1992),"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
3,All_dogs_go_to_heaven(1989),"1\n00:00:12,319 --> 00:00:14,821\nCAPTIONING M..."
4,An_American_tail(1986),"ï»¿1\n00:02:24,080 --> 00:02:26,528\n(INDISTIN..."
...,...,...
110,Warm_bodies(2013),"2\n00:00:26,559 --> 00:00:28,627\n<i>What am I..."
111,Westworld_scenes_of_Dr_Robert_Ford,"1\n00:00:00,000 --> 00:00:21,179\n[Music]\n\n2..."
112,We_are_the_Millers(2013),"1\n00:00:02,400 --> 00:00:03,731\n<i>Oh, my Go..."
113,While_You_Were_Sleeping(1995),"1\n00:02:20,760 --> 00:02:24,720\nLUCY: <i>Oka..."


Во второй таблице, тоже собранной из данных файлов в папке заказчика, но другой, не размеченной по уровню языка, есть названия фильмов и текст субтитров.

In [10]:
df_3 = pd.read_excel('D:/Маша DS ЯП/Проекты/Мастерская 2/English_level/English scores/movies_labels.xlsx')

In [11]:
display(df_3)

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+
...,...,...,...
236,236,Matilda(2022),C1
237,237,Bullet train,B1
238,238,Thor: love and thunder,B2
239,239,Lightyear,B2


In [12]:
df_3.drop(columns='id', axis=1)

,Movie,Level
0,10_Cloverfield_lane(2016),B1
1,10_things_I_hate_about_you(1999),B1
2,A_knights_tale(2001),B2
3,A_star_is_born(2018),B2
4,Aladdin(1992),A2/A2+
...,...,...
236,Matilda(2022),C1
237,Bullet train,B1
238,Thor: love and thunder,B2
239,Lightyear,B2


В третьей таблице, предоставленной самим заказчиком, есть названия фильмов и уровень языка.

Теперь нам предстоит объединить эти данные в одну таблицу.

Столбцом для объединения точно будет название фильма. Объединение df_3 с df_2 не составит труда.

In [22]:
df_2_3 = df_3.merge(df_2, on='Movie', how='outer').drop(columns='id')

In [23]:
len(df_2_3)

250

In [24]:
len(df_2)

115

In [25]:
len(df_3)

241

In [26]:
df_2_3.isna().sum()

Movie      0
Level      9
Srt      131
dtype: int64

In [27]:
df_2_3.sample(10)

,Movie,Level,Srt
95,The_lion_king(1994),A2/A2+,"1\n00:00:33,000 --> 00:00:40,000\n\n\n2\n00:00..."
170,Suits.Episode 8- Mea Culpa,B2,NaN
231,Suits.S03E07.HDTV.x264-mSD,C1,NaN
65,Oceans_Eleven(2001),C1,"1\n00:00:04,393 --> 00:00:07,056\n<i>GUARD: On..."
130,Seven.Worlds.One.Planet.S01E07.2160p.BluRay.Re...,B1,NaN
216,Suits S04E08 EngSub,C1,NaN
85,The_terminal(2004),"A2/A2+, B1","1\n00:00:26,200 --> 00:00:27,918\n<font color=..."
184,Suits.S02E01.HDTV.x264-AVS,B2,NaN
166,Suits.Episode 4- No Puedo Hacerlo,B2,NaN
192,Suits.S02E09.HDTV.x264-ASAP,B2,NaN


In [28]:
df_all = df_2_3.merge(df_1, on='Movie', how='outer')

In [29]:
df_all

,Movie,Level_x,Srt_x,Level_y,Srt_y
0,10_Cloverfield_lane(2016),B1,"1\n00:00:55,279 --> 00:01:07,279\n<font color=...",NaN,NaN
1,10_things_I_hate_about_you(1999),B1,"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0...",NaN,NaN
2,A_knights_tale(2001),B2,"1\n00:00:15,089 --> 00:00:21,229\nResync: Xenz...",NaN,NaN
3,A_star_is_born(2018),B2,"1\n00:00:17,610 --> 00:00:22,610\n- <i><font c...",NaN,NaN
4,Aladdin(1992),A2/A2+,"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com...",NaN,NaN
...,...,...,...,...,...
285,Virgin.River.S01E06.INTERNAL.720p.WEB.x264-STRiFE,NaN,NaN,B2,"1\n00:00:18,852 --> 00:00:19,852\nHey.\n\n2\n0..."
286,Virgin.River.S01E07.INTERNAL.720p.WEB.x264-STRiFE,NaN,NaN,B2,"1\n00:00:10,468 --> 00:00:13,178\nAre you sure..."
287,Virgin.River.S01E08.INTERNAL.720p.WEB.x264-STRiFE,NaN,NaN,B2,"1\n00:00:07,382 --> 00:00:10,012\nTwo IVs in p..."
288,Virgin.River.S01E09.INTERNAL.720p.WEB.x264-STRiFE,NaN,NaN,B2,"1\n00:00:16,474 --> 00:00:18,024\nOh.\n\n2\n00..."


In [30]:
df_all.isna().sum()

Movie        0
Level_x     49
Srt_x      171
Level_y    127
Srt_y      127
dtype: int64

In [31]:
df_all['Level']=df_all['Level_x']+df_all['Level_y']

In [32]:
df_all.isna().sum()

Movie        0
Level_x     49
Srt_x      171
Level_y    127
Srt_y      127
Level      167
dtype: int64

In [33]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 290 entries, 0 to 289
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Movie    290 non-null    object
 1   Level_x  241 non-null    object
 2   Srt_x    119 non-null    object
 3   Level_y  163 non-null    object
 4   Srt_y    163 non-null    object
 5   Level    123 non-null    object
dtypes: object(6)
memory usage: 15.9+ KB


In [34]:
df_all['Level'].unique()

array([nan, 'A2A2', 'B1B1', 'B2B2', 'C1C1'], dtype=object)

In [35]:
df_all['Level_x'].unique()

array(['B1', 'B2', 'A2/A2+', 'C1', 'B1, B2', 'A2/A2+, B1', 'A2', nan],
      dtype=object)

In [36]:
df_all['Level_y'].unique()

array([nan, 'A2', 'B1', 'B2', 'C1'], dtype=object)